# Write-up and code for Assignment 4 - Week 2

### To do:
- Write code for Policy Evaluation (tabular) algorithm
- Write code for Policy Iteration (tabular) algorithm
- Write code for Value Iteration (tabular) algorithm
- Those familiar with function approximation (deep networks, or simply linear in featues) can try writing code for the above algorithms with function approximation (a.k.a. Approximate DP)

In [7]:
def policy_eval(mdp: MDP, policy: Policy, n_iter: int) -> state_value_function:
    # implementation of policy evaluation
    vf = {s: 0. for s in mdp.States}
    for i in range(n_iter):
        new_vf = {}
        for s in mdp.States:
            new_vf[s] = 0
            for a in policy[s]:
                new_vf[s] += policy[s][a]*mdp.R[s][a]
                for sp in mdp.P[s][a]:
                    new_vf[s] += policy[s][a]*mdp.gamma*mdp.P[s][a][sp]*vf[sp]
        vf = new_vf
    return vf

## Appendix code
Code needed from previous assignments

In [1]:
from typing import TypeVar

S = TypeVar('S')
A = TypeVar('A')

In [2]:
from typing import NamedTuple, Any, Dict, Tuple, Set
import numpy as np

class MP(NamedTuple):
    States: Set[S]
    P: Dict[S, Dict[S, float]]
        
        
class MRP(NamedTuple):
    # assumes that the reward is just a function of the current state
    mp: MP
    R: Dict[S, float]
    gamma: float
        
        
class MRP(NamedTuple):
    # assumes that the reward is a function of the transition
    mp: MP
    R: Dict[S, Dict[S, float]]
    gamma: float

In [6]:
class MDP(NamedTuple):
    States: Set[S]
    # the transitions depend on s, a, and s'
    # mapping from a state to a mapping of an action to a mapping of a state to a float (probability)
    P: Dict[S, Dict[A, Dict[S, float]]]
    Actions: A
    # reward is a function of the current state,  and the action
    R: Dict[S, Dict[A, float]]
    gamma: float
        
        
class MDP(NamedTuple):
    States: Set[S]
    # the transitions depend on s, a, and s'
    # mapping from a state to a mapping of an action to a mapping of a state to a float (probability)
    P: Dict[S, Dict[A, Dict[S, float]]]
    Actions: A
    # reward is a function of the current state s, the action a, and the next state sp
    R: Dict[S, Dict[A, Dict[S, float]]]
    gamma: float
        
        
class Policy(NamedTuple):
    # state to action to a probability
    pi: Dict[S, Dict[A, float]]
        

class state_value_function(NamedTuple):
    vf: Dict[S,  float]
        
        
class action_value_function(NamedTuple):
    vf: Dict[S, Dict[A, float]]